## Zapier Natural Language Actions API
\
Full docs here: https://nla.zapier.com/api/v1/dynamic/docs

**Zapier Natural Language Actions** gives you access to the 5k+ apps, 20k+ actions on Zapier's platform through a natural language API interface.

NLA supports apps like Gmail, Salesforce, Trello, Slack, Asana, HubSpot, Google Sheets, Microsoft Teams, and thousands more apps: https://zapier.com/apps

Zapier NLA handles ALL the underlying API auth and translation from natural language --> underlying API call --> return simplified output for LLMs. The key idea is you, or your users, expose a set of actions via an oauth-like setup window, which you can then query and execute via a REST API.

NLA offers both API Key and OAuth for signing NLA API requests.

1. Server-side (API Key): for quickly getting started, testing, and production scenarios where LangChain will only use actions exposed in the developer's Zapier account (and will use the developer's connected accounts on Zapier.com)

2. User-facing (Oauth): for production scenarios where you are deploying an end-user facing application and LangChain needs access to end-user's exposed actions and connected accounts on Zapier.com

This quick start will focus on the server-side use case for brevity. Review [full docs](https://nla.zapier.com/api/v1/dynamic/docs) or reach out to nla@zapier.com for user-facing oauth developer support.

This example goes over how to use the Zapier integration with a `SimpleSequentialChain`, then an `Agent`.
In code, below:

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# get from https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = ""

# get from https://nla.zapier.com/demo/provider/debug (under User Information, after logging in): 
os.environ["ZAPIER_NLA_API_KEY"] = ""

In [1]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, TransformChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.tools.zapier import ZapierNLAListActions, ZapierNLARunAction
from langchain.utilities.zapier import ZapierNLAWrapper

In [3]:
## step 0. expose gmail 'find email' and slack 'send channel message' actions

# first go here, log in, expose (enable) the two actions: https://nla.zapier.com/demo/start -- for this example, can leave all fields "Have AI guess"
# in an oauth scenario, you'd get your own <provider> id (instead of 'demo') which you route your users through first

actions = ZapierNLAWrapper().list()

In [ ]:
## step 1. gmail find email

GMAIL_SEARCH_INSTRUCTIONS = "Grab the latest email from Bryan Helmig"

def nla_gmail(inputs):
    action = next((a for a in actions if a["description"].startswith("Gmail: Find Email")), None)
    return {"email_data": ZapierNLARunAction(action_id=action["id"]).run(inputs["instructions"])}
gmail_chain = TransformChain(input_variables=["instructions"], output_variables=["email_data"], transform=nla_gmail)

In [ ]:
## step 2. generate draft reply

template = """You are an assisstant who drafts replies to an incoming email. Output draft reply in plain text (not JSON).

Incoming email:
{email_data}

Draft email reply:"""

prompt_template = PromptTemplate(input_variables=["email_data"], template=template)
reply_chain = LLMChain(llm=OpenAI(temperature=.7), prompt=prompt_template)

In [ ]:
## step 3. send draft reply via a slack direct message

SLACK_HANDLE = "@knoop"

def nla_slack(inputs):
    action = next((a for a in actions if a["description"].startswith("Slack: Send Direct Message")), None)
    instructions = f'Send this to {SLACK_HANDLE} in Slack: {inputs["draft_reply"]}'
    return {"slack_data": ZapierNLARunAction(action_id=action["id"]).run(instructions)}
slack_chain = TransformChain(input_variables=["draft_reply"], output_variables=["slack_data"], transform=nla_slack)

In [ ]:
## finally, execute

overall_chain = SimpleSequentialChain(chains=[gmail_chain, reply_chain, slack_chain], verbose=True)
overall_chain.run(GMAIL_SEARCH_INSTRUCTIONS)

## Example with Agent
Zapier tools can also be run with an agent. See the example below.

In [3]:
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit

In [4]:
llm = OpenAI(temperature=0)
zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.get_tools(), llm, agent="zero-shot-react-description", verbose=True)

In [6]:
agent.run("Summarize the last email I received regarding Silicon Valley Bank. Send the summary to the #general Slack channel.")



> Entering new AgentExecutor chain...
 I need to find the email and summarize it.
Action: Gmail: Find Email
Action Input: "Find the last email from Silicon Valley Bank"
Observation: {"from__name": "Silicon Valley Bank, N.A.", "subject": "Silicon Valley Bank \u2013 Open for business", "from__email": "sreply@svb.com", "body_plain": "Dear Clients: Silicon Valley Bank, N.A. is open & conducting business as usual. I'm the new CEO starting today & recognize the past few days have been challenging. The FDIC transferred all deposits & assets to a bridge bank to protect depositors. All deposits are protected & new deposits are too. Wire payments entered on 3/9-10 have been cancelled, so reinitiate them. We're conducting business in the US & expect to resume cross-border soon. We appreciate your patience as we serve you. I come with experience in similar situations & an appreciation for the innovation economy. We look to restore your confidence & support you & your companies. Thank you & best 

'I have sent a summary of the last email from Silicon Valley Bank, N.A. to the #general Slack channel.'